# Similarity

---------------

__Create__: Yuan.Sh

__Date__: 2021-11-30 22:03:57

__Update_1__: 2022-06-14 16:41:23

__Email__: yuansh3354@163.com

__Blog__: https://blog.csdn.net/qq_40966210

__Address__:Fujian Medical University

---------------

## Summary of data 

| Cell_subType    | Count | Cancer or Normal |  |
| --------------- | ----- | ---------------- |  |
| AT1             | 530   | Normal           |  |
| AT2             | 2020  | Normal           |  |
| Ciliated        | 654   | Normal           |  |
| Club            | 439   | Normal           |  |
| Undetermined    | 60    | NA               |  |
| tS1             | 3270  | Tumor            |  |
| tS2             | 3018  | Tumor            |  |
| tS3             | 64    | Tumor            |  |
| nan             | 1628  | NA               |  |
| Malignant cells | 24784 | Tumor            |  |

---

In the [article of Data](https://doi.org/10.1038/s41467-020-16164-1), authors use infercnv to define which cell is cancer cell. If any cell has copy number variation(use some threshold, see article), that is considered as a cancer cell, otherwise it is Na.

However, generally speaking, we always treat no-CNV cell as Normal cell. Because that the threshold is not suitable for anyother datasets, and also, not everyone konws how to choose suitable threshold.

__So, in the following study, we drop-out the `NA` value, as the article label. and use `Normal` replace `NA` as inferCNV label__

> In this section, we just only evaluated the `Similarity` between __artivle-label__ and __CaSee-label__

Import `GSE131907_epi_meta.csv` which contained all epithelial cell.

__All cells annotated by Kim. et al__

In [1]:
### 常规设置
import os
import math
import itertools
import warnings
import numpy as np
import glob 
import pandas as pd
import pytorch_lightning as pl
from collections import Counter
import multiprocessing
import matplotlib.pyplot as plt
from PIL import Image
import scanpy as sc
import re 
meta = pd.read_csv('../GSE131907_epi_meta.csv',index_col=0)


Import `merge_data_predict_Integration_.csv` which was the output of CaSee

In [2]:
predict = pd.read_csv('../merge_data_h5_file/merge_data_predict_Integration_.csv',index_col=0)
meta = meta.dropna()
ids = meta.index
predict = predict.loc[ids]


__Calculation of the similarity of CaSee's results with those of Kim. et al.__

In [4]:
sum(predict['scale_predict'] == meta['article_label']) / predict.shape[0]
from sklearn.metrics import confusion_matrix
confusion_matrix(predict['scale_predict'], meta['article_label'], labels=['Cancer','Normal'])

0.9413875254743248

array([[29644,   550],
       [ 1492,  3153]])

##### similarity group by tissue

In [8]:
meta['predict'] = predict['scale_predict']
df = meta.groupby(['Sample_Origin','article_label','predict']).count()
df = pd.DataFrame(df['Cell_subtype'])
df.columns = ['Count']
df = df.loc[Counter(meta['Sample_Origin']).keys()]
ids = [val for val in Counter(meta['Sample_Origin']).values() for i in range(2)]

df['similarity']  = df['Count'] / ids
df.to_csv('~/Desktop/GSE131907_tiss_similarity.csv')

df

Count  similarity
Sample_Origin article_label predict                   
nLung         Normal        Cancer     550    0.148528
                            Normal    3153    0.851472
tLung         Cancer        Cancer    5764    0.907431
                            Normal     588    0.092569
tL/B          Cancer        Cancer    5981    0.934531
                            Normal     419    0.065469
mLN           Cancer        Cancer    2869    0.968929
                            Normal      92    0.031071
mBrain        Cancer        Cancer   15030    0.974519
                            Normal     393    0.025481